# Третий этап обучения

## Решаем задачу сегментации, повышая разрешение изображения до 1024x1024

In [1]:
%%capture

# Загрузка датасета
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    !cp /content/drive/MyDrive/crimea/train_crimea.zip /content/
    !unzip -oq /content/train_crimea.zip

    !cp /content/drive/MyDrive/crimea/masks2.zip /content/
    !unzip -oq /content/masks2.zip

    !cp /content/drive/MyDrive/crimea/test_crimea.zip /content/
    !unzip -oq /content/test_crimea.zip

    !mkdir /content/test_masks

    !python -m pip install --upgrade pip
    !pip install -U segmentation_models_pytorch

    COLAB      = True
    ROOT       = '/content/'
    ROOT_DRIVE = '/content/drive/MyDrive/crimea/'

except:
    COLAB      = False
    ROOT       = './'
    ROOT_DRIVE = './'


In [2]:
!nvidia-smi

Thu Nov 24 07:01:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   28C    P0    44W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
import glob
import random
import warnings
from pathlib import Path
from typing import List
from datetime import datetime

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
from sklearn.model_selection import train_test_split


SEED = 1
STAGE = 3
DEVICE = 'cuda'
EPOCHS = 24
MAX_LR = 2e-4
BATCHSIZE = 6

GRAD_ACC_STEPS = 8

ARCH = 'DeepLabV3Plus'
ENCODER = 'tu-xception71'
PREVIOUS= f'stage_{STAGE - 1}'
VERSION = f'stage_{STAGE}'

warnings.filterwarnings("ignore", 'User provided device_type of \'cuda\', but CUDA is not available. Disabling')
warnings.filterwarnings("ignore", 'torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.')

In [4]:
model = smp.DeepLabV3Plus(encoder_name=ENCODER,
                          encoder_weights=None, classes=4)

model.load_state_dict(torch.load(f"{ROOT_DRIVE}{ARCH}_{ENCODER}_{PREVIOUS}.pth",
                                 map_location='cpu'))

scaler = torch.cuda.amp.GradScaler()

In [5]:
ROOT = Path(ROOT)

train_image_path = ROOT / "images"
train_mask_path = ROOT / "masks2"

ALL_IMAGES = sorted(train_image_path.glob("*.png"))
ALL_MASKS = sorted(train_mask_path.glob("*.png"))

assert len(ALL_IMAGES) == len(ALL_MASKS)

print(len(ALL_IMAGES))

3500


In [6]:
def seed_everything(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark     = False

seed_everything(SEED + STAGE)

In [7]:
transform_train = A.Compose([
    A.ShiftScaleRotate(shift_limit  = 0.05,
                       scale_limit  = 0.05,
                       rotate_limit = 15,
                       p=1.0),
    A.Resize(1024, 1024, interpolation=cv2.INTER_AREA),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.PiecewiseAffine(p=0.5),
    A.OneOf([
        A.CLAHE(clip_limit=2, p=1),
        A.Sharpen(p=1),
        A.Emboss(p=1),
    ], p=0.3),
    A.ColorJitter(hue=0.03, p=1.0),
    A.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

transform_val = A.Compose([
    A.Resize(1024, 1024, interpolation=cv2.INTER_AREA),
    A.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

In [8]:
class SegmentationDataset(Dataset):
    def __init__(
        self,
        images: List[Path],
        masks: List[Path] = None,
        transforms=None
    ) -> None:
        self.images = images
        self.masks = masks
        self.transforms = transforms

    def __len__(self) -> int:
        return len(self.images)

    def __getitem__(self, idx: int) -> dict:
        image_path = self.images[idx]
        image = cv2.imread(str(image_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, _ = image.shape
        
        result = {"image": image, "hw": [h, w]}
        
        if self.masks is not None:
            mask = cv2.imread(str(self.masks[idx]), 0)
            result["mask"] = mask
        
        if self.transforms is not None:
            result = self.transforms(**result)
        
        result["filename"] = image_path.name

        return result

In [9]:
all_images = np.asarray(ALL_IMAGES)
all_masks  = np.asarray(ALL_MASKS)

train_img, test_img, train_mask, test_mask = train_test_split(all_images, all_masks,
                                                              random_state=SEED, test_size=0.05)

dataset_train = SegmentationDataset(train_img, masks=train_mask, transforms=transform_train)
dataset_val   = SegmentationDataset(test_img, masks=test_mask, transforms=transform_val)

In [10]:
model.train()
model.to(DEVICE)

dice      = smp.losses.DiceLoss(mode='multiclass', classes=[1, 2, 3],
                                log_loss=False, from_logits=True,
                                smooth=1.0, eps=1e-08)
criterion = torch.nn.BCEWithLogitsLoss()

iou       = smp.losses.JaccardLoss(mode='multiclass',
                                   classes=[1, 2, 3], log_loss=False,
                                   smooth=1.0)

## Start train

In [11]:
loader_train = DataLoader(
  dataset_train,
  batch_size=BATCHSIZE,
  shuffle=True,
  num_workers=2,
  drop_last=True,
)

loader_val = DataLoader(
  dataset_val,
  batch_size=BATCHSIZE,
  shuffle=False,
  num_workers=2,
  drop_last=False,
)

seed_everything(SEED + STAGE)

optimizer = torch.optim.AdamW(model.parameters(),
                              lr=MAX_LR,
                              weight_decay=1e-2)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                                       EPOCHS // 2,
                                                       eta_min=1e-8)

print('start at', datetime.now().strftime("%H:%M:%S"))
best_metric = 0.0
best_cnt    = 0
optimizer.zero_grad()
for epoch in range(EPOCHS):
    losses     = []
    losses_bce = []
    losses_dc  = []

    model.train()
    torch.cuda.empty_cache()

    for i, batch in enumerate(loader_train, start=1):
        mask = batch['mask'].to(DEVICE)
        with torch.cuda.amp.autocast(enabled=True):
            pred = model.forward(batch['image'].to(DEVICE))
            
            dc = dice(pred, mask.long())
            y_pred = (pred.argmax(dim=1) > 0).float()
            y_true = (mask > 0).float()
            bce = criterion(y_pred, y_true)
            
            loss = dc + bce * 0.1
        
        losses.append(loss.item() / GRAD_ACC_STEPS)
        losses_bce.append(bce.item() / GRAD_ACC_STEPS)
        losses_dc.append(dc.item() / GRAD_ACC_STEPS)

        loss = loss / GRAD_ACC_STEPS
        scaler.scale(loss).backward()

        if i % GRAD_ACC_STEPS == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        
    scheduler.step()
    print(datetime.now().strftime("%H:%M:%S"),
          f'epoch {epoch:02d} loss {np.mean(losses):.3f} bce {np.mean(losses_bce):.3f} '
          f'dice {np.mean(losses_dc):.3f} lr={optimizer.param_groups[0]["lr"]:.8f}')

    val_losses     = []
    val_losses_bce = []
    val_losses_dc  = []
    metrics        = []
    model.eval()
    torch.cuda.empty_cache()
    with torch.no_grad():
        for batch in loader_val:
            pred = model.forward(batch['image'].to(DEVICE))
            mask = batch['mask'].to(DEVICE)

            dc = dice(pred, mask.long())
            y_pred = (pred.argmax(dim=1) > 0).float()
            y_true = (mask > 0).float()
            bce = criterion(y_pred, y_true)

            loss = dc + bce * 0.2
            metric = 1 - iou(pred, mask.long()).item()

            val_losses.append(loss.item())
            val_losses_bce.append(bce.item())
            val_losses_dc.append(dc.item())

            metrics.append(metric)

    print(datetime.now().strftime("%H:%M:%S"), f'valid    loss {np.mean(val_losses):.3f} bce {np.mean(val_losses_bce):.3f} '
          f'dice {np.mean(val_losses_dc):.3f} metric {np.mean(metrics):.3f}')

    best_cnt += 1
    if best_metric <= np.mean(metrics):
        best_cnt    = 0
        best_metric = np.mean(metrics)
        torch.save(model.state_dict(), f"{ROOT_DRIVE}{ARCH}_{ENCODER}_{VERSION}.pth")
        print('Save model')
    if best_cnt > 3:
        best_cnt = 0
        model.load_state_dict(torch.load(f"{ROOT_DRIVE}{ARCH}_{ENCODER}_{VERSION}.pth", map_location='cpu'))
        print('Reload model')


torch.cuda.empty_cache()
print('done at', datetime.now().strftime("%H:%M:%S"))

start at 07:01:28
07:30:21 epoch 00 loss 0.035 bce 0.083 dice 0.026 lr=0.00019659
07:30:27 valid    loss 0.321 bce 0.662 dice 0.189 metric 0.709
Save model
08:00:13 epoch 01 loss 0.034 bce 0.083 dice 0.025 lr=0.00018660
08:00:19 valid    loss 0.325 bce 0.662 dice 0.192 metric 0.705
08:29:08 epoch 02 loss 0.034 bce 0.083 dice 0.025 lr=0.00017071
08:29:14 valid    loss 0.327 bce 0.662 dice 0.194 metric 0.703
08:57:30 epoch 03 loss 0.033 bce 0.083 dice 0.024 lr=0.00015000
08:57:36 valid    loss 0.319 bce 0.662 dice 0.187 metric 0.712
Save model
09:27:44 epoch 04 loss 0.033 bce 0.083 dice 0.024 lr=0.00012589
09:27:50 valid    loss 0.321 bce 0.662 dice 0.189 metric 0.711
09:57:21 epoch 05 loss 0.033 bce 0.083 dice 0.024 lr=0.00010000
09:57:27 valid    loss 0.321 bce 0.662 dice 0.188 metric 0.712
10:27:03 epoch 06 loss 0.032 bce 0.083 dice 0.024 lr=0.00007412
10:27:08 valid    loss 0.315 bce 0.662 dice 0.183 metric 0.719
Save model
10:56:40 epoch 07 loss 0.032 bce 0.083 dice 0.024 lr=0.00005

## Делаем прогноз

In [12]:
model.load_state_dict(torch.load(f"{ROOT_DRIVE}{ARCH}_{ENCODER}_{VERSION}.pth",
                                 map_location='cpu'))
model.eval()
model.to(DEVICE)

transform_test = A.Compose([
    A.Resize(1024, 1024, interpolation=cv2.INTER_AREA),
    A.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

test_image_path = Path(ROOT) / "test"

with torch.no_grad():
    torch.cuda.empty_cache()
    for i, image_path in enumerate(sorted(test_image_path.glob("*.png"))):
        if i % 100 == 0:
            print(f"{i:04d} {image_path}")

        img = cv2.imread(str(image_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape
        
        img  = transform_test(image=img)['image'].unsqueeze(0).to(DEVICE)
        pred = model.forward(img)
        mask = torch.argmax(pred, dim=1).squeeze().cpu().numpy()
        mask = cv2.resize(mask, (w, h), 0, 0, interpolation=cv2.INTER_NEAREST)
                
        cv2.imwrite(f'{str(ROOT)}/test_masks/{image_path.name}', mask)

print('done')

0000 /content/test/00388128-357d-44bb-a630-5e8856e4dcb0.png
0100 /content/test/0c2d854d-01e5-4b6e-ba78-1b443edcd784.png
0200 /content/test/1af2e8a6-c3e3-4e80-b6b9-1ef8d693a57f.png
0300 /content/test/2aadb686-e5ec-4c3d-bcb1-b76e3030c62e.png
0400 /content/test/3aa52680-b2f0-487a-b6c7-e38050a4a04e.png
0500 /content/test/4ac6f8a4-957b-4be7-aee6-f40019dbe356.png
0600 /content/test/58b3ccd9-4c6d-477b-8dd0-6b09dfe73d5d.png
0700 /content/test/619d426b-4a34-4d46-ad9e-45a8ece730dd.png
0800 /content/test/6f431a1c-0a6c-480d-bffd-629b0fbd7a0d.png
0900 /content/test/7bec62bf-f5ef-4d0e-b525-0f30aca7f60f.png
1000 /content/test/8b3e4e4c-3499-4fad-9f87-02073fed2fd2.png
1100 /content/test/9b793582-e744-450c-a142-e34d0a7cb2ec.png
1200 /content/test/b32baacd-bf29-49e9-b2eb-9a718ea4496d.png
1300 /content/test/bf3e649d-db84-433e-9f4c-82b7f040c226.png
1400 /content/test/de171555-fcde-48e1-9d11-b5f16a3997a8.png
done


In [13]:
if COLAB:
    %cd /content/test_masks/
    !zip -q ../test_masks_3.zip *.png
    !cp ../test_masks_3.zip /content/drive/MyDrive/crimea/

/content/test_masks
